<script type="text/x-mathjax-config">
  MathJax.Hub.Config({
    tex2jax: {
      inlineMath: [['$','$'], ['\\(','\\)']],
      displayMath: [['$$','$$'], ['\\[','\\]']]
    },
    TeX: {
      extensions: ["color.js"]
    },
    "HTML-CSS": {
      availableFonts: ["TeX"],
      linebreaks: { automatic: true }
    }
  });
</script>

<script src="https://cdn.jsdelivr.net/npm/mathjax@2/MathJax.js?config=TeX-AMS-MML_HTMLorMML" type="text/javascript"></script>

<style>
@media all {
  body {
    font-weight: 700;
  }
  /* Rende le formule MathJax più spesse */
  .MathJax, .MathJax_Display {
    font-weight: bold; /* Aumenta lo spessore delle formule */
  }
  .bold-text {
    font-weight: bold;
  }
  .heavy-text {
    font-weight: 700;
  }
}
</style>

# <span style="color: red;">TMPSVM</span>


---

## Take a look at the data

In [1]:
import matplotlib.pyplot as plt
import math

def pie_percent(data_set, column_name, subplot_index, num_rows):
    # Plot a pie chart of the percentage of a specific column
    value_counts = data_set[column_name].value_counts()
    percent = 100 * value_counts / len(data_set[column_name])
    labels = [f'{label}: {percent_val:.1f}%' for label, percent_val in zip(value_counts.index, percent)]
    plt.subplot(num_rows, 2, subplot_index)
    plt.pie(percent, labels=labels, autopct=None, startangle=90, colors=plt.cm.Paired.colors)
    plt.axis('equal')  # Ensure pie chart is drawn as a circle.
    plt.title(f'Percentage distribution of {column_name}')

def take_a_look(clean_df, X_encoded, Y_encoded):
    # Plot the pie-distribution of each column of the dataset
    names = clean_df.columns
    num_columns = clean_df.shape[1]
    num_rows = math.ceil((num_columns + 1) / 2)  
    plt.figure(figsize=(12, num_rows * 2)) 
    for i in range(num_columns):
        pie_percent(clean_df, names[i], i + 1, num_rows)
    plt.tight_layout()
    plt.show()
    # Print the min and max values for each variable
    x_names = X_encoded.columns
    for i in range(X_encoded.shape[1]):
        print(f"Value of {x_names[i]}: min = {X_encoded.iloc[:, i].min()}, max = {X_encoded.iloc[:, i].max()}")



---

## Data splitting and normalizing
<span style="font-size: 14px;">

Splitting the dataset is essential for validating machine learning models, as it helps ensure that the model generalizes well to unseen data. By dividing the data into training (70%) and validation sets (30%), we can assess the model's performance accurately and mitigate the risk of overfitting. This process allows for better tuning of hyperparameters and enhances the overall reliability of the model.

All features are normalized to ensure that all variables contribute equally to the model, preventing features with larger scales from dominating the learning process. It also helps improve the convergence speed of optimization algorithms.

In [2]:
import pandas as pd
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split

# Split the dataset into training and validation sets (75% training and 25% validation)
def data_splitting(X_encoded, Y_encoded,classes):
    X_train, X_val, Y_train, Y_val = train_test_split(
        X_encoded.values, 
        Y_encoded.values, 
        test_size=0.25, # as suggest in the paper
        random_state=42, # Using the intergalatic guide to split the dataset
        stratify=Y_encoded.values  # Keep the class distribution intact
    )
    m_tot = X_encoded.shape[0]  # Total number of data points
    m_train = X_train.shape[0]  # Training set dimensions (number of data points)
    m_val = X_val.shape[0]  # Validation set dimensions (number of data points)
    n = X_train.shape[1]  # Number of features

    print(f"Features: {n} \n"+
            f"Classes: {classes} \n" +
            f"Dataset row: {m_tot} \n" +
            f"Dataset training row: {m_train} \n" +
            f"Dataset validation row: {m_val}")

    # Normalize the data from 0 to 1
    from sklearn.preprocessing import MinMaxScaler
    scaler = MinMaxScaler()
    X_train_scaled = scaler.fit_transform(X_train)
    X_val_scaled = scaler.transform(X_val)
    X_train=X_train_scaled
    X_val=X_val_scaled
    return X_train, X_val, Y_train, Y_val



---

## Support Vector Machine <span style="font-size: 18px;">(brief introduction)</span>
<span style="font-size: 14px;">

- <span style="color: red;">Hard Margin SVM</span> performs classification by identifying a hyperplane that cleanly separates the two classes of $y$ (no positive values are allowed on the negative side and vice versa)

  <img src="HSVM.jpg" alt="Hard SVM" width="300" height="180">

- <span style="color: red;">Soft Margin SVM</span> extends hard SVM by allowing some misclassifications through slack variables ($\xi^+$ and $\xi^-$). The objective is to minimize the weight vector norm and the penalty for misclassifications, controlled by a regularization parameter $C$. This flexibility helps handle overlapping classes and noise in the data.

  <img src="SSVM.jpg" alt="Soft SVM" width="300" height="180">

- <span style="color: red;">Non Linear SVM</span> uses kernel functions to project data into a higher-dimensional space, enabling the identification of complex decision boundaries. This approach allows for efficient classification of non-linearly separable data without explicitly transforming it.<br>

  <img src="NLSVM.jpg" alt="Non Linear SVM" width="300" height="180">

- <span style="color: red;">Twin Parametric Margin SVM</span>: According to the classical SVM mentioned above, many SVM variants have been devised in the literature. Among them, the TPMSVM detects two non-parallel hyperplanes (or two hyperspheres) to separate the two classes of $y$, such that training observations of the other class are as far as possible from the opposing classifier. This approach reduces computational complexity since each hyperplane (classifier) is the solution to a part of the overall problem.  <br>

  <img src="TPMSVM.jpg" alt="TPMSVM" width="600" height="180">

- <span style="color: red;">Multiclass TPMSVM</span>: To extend this approach to the multiclass problem, it is possible to adopt the OvR (One versus Rest) strategy. In this project, the linear multiclass TPMSVM will not be applicable, so the Non-Linear multiclass TPMSVM will be used.All the model formulations are derived from the paper "A Robust Twin Parametric Margin Support Vector Machine for Multiclass Classification" by Renato de Leone, Francesca Maggioni, and Andrea Spinelli: https://arxiv.org/pdf/2306.06213 <br>

  <img src="Linear Multiclass TPMSVM.jpg" alt="Linear Multiclass TPMSVM" width="300" height="180">
  <img src="Non Linear Multiclass TPMSVM.jpg" alt="Non Linear Multiclass TPMSVM" width="300" height="180">



---
## Multiclass TPMSVM 
<span style="font-size: 14px;">

From a conceptual point of view, each data point can be visualized as a point in an $n$-dimensional space ($n$ = number of features $x$), associated with a label $y$. Classification through SVM assumes that these points form "clouds" of data with the same label, separated by surfaces (either flat or curved).<br>
 - The scalar $\textcolor{red}{C}$ is the number of "clouds" corresponding to the number of classes.$\\[2pt]$
 - The scalar $\textcolor{red}m$ is the total number of points (Dataset rows); the scalars $m_c$ and $m_o$ denotes the number of observations beloging to class $c$ and the remaining observations according to $m = m_c + m_o \\[2pt]$
 - The scalar $\textcolor{red}n$ is the features' number i.e. number of the dimensions of the feature space (Dataset columns).$\\[2pt]$
 - Thus $\textcolor{red}{\{x^i, y_i\}^m_{i=1}}$ is the set of training samples, with $y_i \in \{1,...,C\} \\[2pt]$
 - The matrix $\textcolor{red}{X_c \in \mathbb{R}^{n \times m_c}}$ represents all training data points of class c. Similarly, for matrix $\textcolor{red}{X_o \in \mathbb{R}^ {n \times m_o}} \\[2pt]$
 - The corresponding index sets are $\textcolor{red}{\mathcal{X}_c }$ and $\textcolor{red}{\mathcal{X}_o}$, respectively, and $\mathcal{X} := \mathcal{X}_c \cup \mathcal{X}_o \\[2pt]$
 - For each class $c = 1,...,C$ we aim to find the best separating hyperplanes $H_c$ definited by the equation $\textcolor{red}{w_c^T x + \theta_c = 0}$ where the vector $\textcolor{red}{w_c \in \mathbb{R}^n}$ is the normal vector of $c$-th hyperplanes and the scalar $\theta_c \in \mathbb{R}$ is the bias (offset from the origin in feature space) of the $c$-th hyperplanes.$\\[2pt]$
 - The scalar $\textcolor{red}{\nu_c}$ is the regularization parameter that regulates the width of the margin between classes and thus affects the stability and robustness of the classifier. An increase in $\nu_c$ leads to a wider margin, represented by a greater distance between the hyperplanes, while a decrease allows for greater flexibility, reducing that distance.$\\[2pt]$
 - The scalar $\textcolor{red}{\alpha_c}$ is the regularization parameter of the "thickness" of the margin of the c-th hyperplane means that it controls how tolerant the model is towards observations near the margin. If $\alpha_c$ is high, the model tends to be more tolerant towards observations that are close to or slightly violate the margin. In other words, a higher $\alpha_c$ allows for a "thicker" margin, accepting more points near or on it, which can improve generalization in some cases. Conversely, a lower $\alpha_c$ reduces tolerance towards these points, aiming to keep the margin "narrower" and forcing a sharper separation, but at the risk of being less flexible towards noisy or complex data.$\\[2pt]$
 - The vector $\textcolor{red}{\xi_c \in \mathbb{R}^{m_c}}$ is the slack vector represent the amount of margin violation that a given point introduces into the model:
    - $\xi_i = 0$ the point $x^i$ is correctly classified and is at least the required distance from the margin.$\\[2pt]$
    - $0 < \xi_i \le 1$ the point $x^i$ is correctly classified but falls within the margin, meaning it is closer to the hyperplane than desired.$\\[2pt]$
    - $ \xi_i > 1 $ the point $x^i$ is misclassified, meaning it is on the wrong side of the separating hyperplane.$\\[2pt]$
 - The vector $\textcolor{red}{e_c \in \mathbb{R}^{m_c}}$ if filled with ones.$\\[2pt]$
 - The scalar $\textcolor{red}{\frac{1}{2} \| w_c|^2_2}$ represents the Regularization Term related to the margin of c-th class.
 - The scalar $\textcolor{red}{\frac{\nu_c}{m_o} e^T_{-c} (X^T_o w_c + e_o \theta_c)}$ considers the projections of the observations not belonging to the c-th class, requiring that theese training points are as far as possible from $H_c \\[2pt]$
 - The scalar $\textcolor{red}{\frac{\alpha_c}{m_c} e^T_c \xi_c}$ is the penalty function regarding the total number of misclassified samples of the c-th class $\\[2pt]$

Note that the above definition doesn't account for the $i$-th dimension (number of classes).

\begin{array}{l l}
\text{Primal Problem for each class c = 1,...,C}& \quad & \quad \\[2pt]
\hline \\
\mathcal {R} (w_c) = \frac{1}{2} \|w_c\|^2_2 & \text{  Regulazitation term} \\[2pt]
\mathcal {P}_{other} (w_c,\theta_c) = \frac{\nu_c}{m_o} e^T_o (X^T_o w_c + e_o \theta_c) & \text{ Penalty for other class} \\[2pt]
\mathcal {P}_{target} (\xi_c) = \frac{\alpha_c}{m_c} e^T_c \xi_c & \text { Penalty for target class misclassified}\\[2pt]
J(w_c,\theta_c,\xi_c) =\mathcal {R} + \mathcal {P}_{other} + \mathcal {P}_{target} & \text { Cost Function with c-th class vs Rest}\\[2pt]
\min_{w_c,\theta_c,\xi_c} J(w_c,\theta_c,\xi_c) & \text { Objective Function with c-th class vs Rest}\\[2pt]
X^T_c w_c + e_c \theta_c \ge - \xi_c & \text { Margin Constraints}\\[2pt]
\xi_c \ge 0, \quad \nu_c > 0, \quad \alpha_c > 0 & \text { Non-negativity Constraints}\\[2pt]
\nu_c / \alpha_c < 1 & \text { Fractional Constraint}
\end{array}


<span style="font-size: 14px;">

In the context of the Twin SVM (TPMSVM) for multi-class classification, when transitioning from the primal problem to the dual problem, a separate Lagrangian is defined for each pair of classes that are classified against the rest. Each Lagrangian $\mathcal{L}$ corresponds to the optimization problem associated with that specific class against all other classes. This indicates that multiple dual problems must be solved, each focusing on the separation of a specific class from the others.<br>

The Lagrangian $\mathcal{L}$ will reach its maximum when all the partial derivates $\delta \mathcal{L} / \delta w_c$, $\delta \mathcal{L} / \delta \theta_c$ and $\delta \mathcal{L} / \delta \xi_c$ are equal to zero.
These three equations allow us to express $w_c, \theta_c \text{ and } \xi_c$ as function of $\lambda_c$ leading us to the dual problem.
where $\lambda_c \in \mathbb{R}^{m_c}$ is the Lagrange multiplier vector for c-th class. <br>

Due to the fractional constraint, only the Lagrange multipliers related to the margins appear in the Lagrangian, and not those related to the non-negativity of the slack variables allowing for a focus on the multipliers associated with the margins.

\begin{array}{l l l}
\text{Dual Problem for each class c = 1,...,C} & \text{ LINEAR TPMSVM} \\[2pt]
\hline \\
\mathcal{L} = - \frac{1}{2} \lambda^T_c X^T_c X_c \lambda_c + \frac{\nu_c}{m_o} e^T_o X^T_o X_c \lambda_c & \text{ Lagrange function} \\[2pt]
max_{\lambda_c} \quad \mathcal{L} & \text{ Objective Function}\\[2pt]
e^T_c \lambda_c - \nu_c =0 &  \text{ normalization constraints}\\[2pt]
\alpha_c - m_c \lambda_c \ge 0 &  \text{ limit constraints}\\[2pt]
\lambda_c \ge 0 &  \text{ no negativity constraint}\\[2pt]
\end{array}

To increase the predictive power of the model, allowing situations where training observations are not linearly separable,  the nonlinear version of the TPMSVM approach provides, according to the classical procedure,  input data points mapping to an inner product space $( \mathcal{H}, \langle.,.\rangle_{\mathcal{H}})$ via a feature map $ \varphi: \mathbb {R}^n \to \mathcal {H} $. Following this idea, the c-th separating hypeplanes becomes:

$$ H_c : \langle w_c, \varphi(x)_\mathcal {H} \rangle + \theta_c = 0 $$

Applying the kernel trick, we have the following dual problem:

\begin{array}{l l l}
\text{Dual Problem for each class c = 1,...,C} & \text{ NON LINEAR TPMSVM} \\[2pt]
\hline \\
\mathcal{L} = - \frac{1}{2} \lambda^T_c K(X_c,X_c) \lambda_c + \frac{\nu_c}{m_o} e^T_o K(X_o,X_c) \lambda_c & \text{ Lagrange function} \\[2pt]
max_{\lambda_c} \quad \mathcal{L} & \text{ Objective Function}\\[2pt]
e^T_c \lambda_c - \nu_c =0 &  \text{ normalization constraints}\\[2pt]
\alpha_c - m_c \lambda_c \ge 0 &  \text{ limit constraints}\\[2pt]
\lambda_c \ge 0 &  \text{ no negativity constraint}\\[2pt]
\end{array}

using the following conventions:

$k(x^i, x^j)= \langle \varphi(x^i), \varphi(x^j) \rangle \\[2pt]
K_{cc}=K(X_c, X_c)$ is the kernel matrix $m_c \times m_c$ with entries $k(x^i, x^j)$ for $i,j \in \mathcal {X_c} \\[2pt]
K_{co}=K(X_c, X_o)$ is the kernel matrix $m_c \times m_o$ with entries $k(x^i, x^j)$ for $i \in \mathcal {X_c}$ and $j \in \mathcal {X_o} \\[2pt]
K_{oc}=K(X_o, X_c)$ is the kernel matrix $m_o \times m_c$ with entries $k(x^i, x^j)$ for $i \in \mathcal {X_o}$ and $j \in \mathcal {X_c}\\[2pt]
K_{oo}=K(X_o, X_o)$ is the kernel matrix $m_o \times m_o$ with entries $k(x^i, x^j)$ for $i,j \in \mathcal {X_o} \\[2pt]
K_c = \begin{pmatrix}
K_{cc} & K_{co} \\
K_{co} & K_{oo} \end{pmatrix} \\[2pt]
K_{cc}$ and $K_{cc}$ are simmetric matrices thus $K_{cc} = K_{cc}^T$ and $K_{oo} = K_{oo}^T$

The matrix $K_c $ represents the total kernel matrix for class $c$. 

Computing the gradient and the hessian for $\lambda$ as: <br>

$g_c = \frac{\nabla\mathcal{L}}{\nabla \lambda_c} \in \mathbb{R}^{m_c} = \frac{\nabla}{\nabla \lambda_c} \left(- \frac{1}{2} \lambda^T_c K_{cc} \lambda_c \right) + \frac{\nabla}{\nabla \lambda_c} \left(\frac{\nu_c}{m_o} e^T_o K_{oc}\lambda_c \right) =$
$ - \frac{1}{2} (K_{cc} + K_{cc}^T) \lambda_c + \frac{\nu_c}{m_o} K_{oc}^T e_o =- K_{cc} \lambda_c + \frac{\nu_c}{m_o} K_{oc}^T e_o$

$ H_c = \frac{\nabla^2\mathcal{L}}{\nabla \lambda_c^2} \in \mathbb{R}^{m_c \times m_c}= \frac{\nabla^2 \mathcal{L}}{\nabla \lambda^2} = \frac{\nabla}{\nabla \lambda} \left( - K_{cc} \lambda_c + \frac{\nu_c}{m_o} K_{oc}^T e_o \right) = - K_{cc}^T = - K_{cc} $

The concavity of the problem (since we are seeking a maximum) is determined by $H_c$, which must be negative semi-definite, meaning its eigenvalues must be non-positive. Since $H_c = - K_{cc}$, this implies that $K_{cc}$ must be positive semi-definite. Therefore: <br>

<span style="color: red;">
The concavity of the problem depends only on the dataset and the kernel function used, meaning that the kernel matrix is a constant matrix. Consequently, the problem is a quadratic function with a constant Hessian, allowing Newton’s method to converge in a single step, as guaranteed by the theorem for quadratic functions. The results obtained confirm this, demonstrating that, regardless of the dataset or the initial point, the method reaches the critical point in one iteration</span>

Once it is verified that $H$ is concave, and since all constraints are linear, it's possibile use the following KKT conditions to find  the optimal solution:

\begin{array}{l l l}
\text{KKT Conditions} & \text{ NON LINEAR TPMSVM}\\[2pt]
\hline \\
- K_{cc} \lambda_c + \frac{\nu_c}{m_o} K_{oc}^T e_o = 0 & \text{ Optimization equation}\\[2pt]
e^T_c \lambda_c - \nu_c =0 &  \text{ normalization constraints}\\[2pt]
\alpha_c - m_c \lambda_c \ge 0 &  \text{ limit constraints}\\[2pt]
\lambda_c \ge 0  & \text{ no negativity constraint}\\
\end{array}


<span style="font-size: 14px;">

The presence of constraints limits the region of feasible solutions and, consequently, the values of the objective function and its derivatives (which, in order to satisfy the KKT conditions, must be continuous almost everywhere). It follows that the objective functions that satisfy the KKT conditions are:
- Lipschitz continuous with parameter $B_c \le \| H_c \|_F $ = Frobenius norm of  $H_c = \sqrt{\sum_{i,j} h_{c_{i,j}}^2}$ since $H_c$ is a simmetric matrix $\| H_c \|_F = \sqrt{\sum_{\lambda \in \sigma (H_c)} \lambda_c^2} \\[2pt]$
- Smooth with parameter $L_c = \max (\sigma (H_c)) = \|H_c\| = $ spectral norm of $H_c \\[2pt]$
- Strongly convex with parameter $\mu_c = \min (\sigma (H_c)) \\[2pt]$

If, by solving the dual problem, all the eigenvalues of H have the correct sign, the KKT conditions are necessary and sufficient (CNS), and therefore: <br>
- B = np.linalg.norm(H, 'fro') $\\[2pt]$
- sigma = np.linalg.eigvals(H) $\\[2pt]$
- L = np.max(np.abs(sigma)) $\\[2pt]$
- mu = np.min(np.abs(sigma)) $\\[2pt]$

For the TPMSVM, the primal problem is a minimization problem, so the dual problem is a maximization problem. This requires that all the eigenvalues of $𝐻_c$ must be non-positive, or that all the eigenvalues of $K_{cc}$ must be non-negative.

Since $\lambda_c$ is bounded by the constraint conditions $ \lambda_c \le \frac {\alpha_c}{m_c}$, it follows that $R = \frac {\alpha_c}{m_c}$.

The above statements guarantee that:
- projected GD with a learning rate $\gamma_{0_c} = \frac {1}{L_c}$ ensures convergence to the optimal solution $\\[6pt]$
- the Final Error after $T_c$ iterations is: 
$FE_c = f(x_T) - f(x^*)\le \frac{R_c^2 L_c}{2} \left( 1 - \frac{\mu_c}{L_c} \right)^{T_c} \\[6pt]$
- the Final Error after $T_c$ iterations is $\le \epsilon \quad $ if $\quad T_c \ge \frac {L_c}{\mu_c} \ln \left( \frac {R_c^2 L_c}{2 \epsilon}\right) \quad \text{i.e.} \quad T_c = \mathcal{O} (log (1/\epsilon)) \\[6pt]$ 
- the Nesterov's Theorem defines, for a Lipschitz function, the Final Error after $T_c$ iterations as $\ge \frac{R_c B_c} {2 (1+ \sqrt{T_c+1})}$

Once the objective functions are solved, by applying the <span style= "font-style: italic;"> Karush-Khun-Tucker </span>(KKT) conditions, the optimal solution $(\tilde{w}_c,\tilde{\theta}_c)$ is:

$$ \tilde{w}_c = \sum_{i \in \mathcal{X}_c} \lambda_{c,i} \varphi(x^i) - \frac{\nu_c}{m_o} \sum_{i \in \mathcal{X}_o }\varphi(x^i) \quad \quad \tilde{\theta}_c = -\frac{1}{|\mathcal{N}_c|} \sum_{i \in \mathcal{N}_c} \langle \varphi(x^i), \tilde{w}_c \rangle_{\mathcal{H}} $$

Defining $\tilde\lambda_{c,i} = 
\begin{cases} 
\lambda_{c,i} & \text{if } i \text{ corresponds to support vectors i.e. } 0 < \lambda_{c,i} < \frac{\alpha_c}{m_c} \\ 
0 & \text{otherwise} 
\end{cases} \quad $ <br> it's possible to simplify the formulas in this way:
$$ \tilde{w}_c =\varphi(X_c) \tilde\lambda_c - \frac{\nu_c}{m_o} \varphi(X_o) e_o \quad \quad \tilde\theta_c = - \frac{1}{|\mathcal{N}_c|} \sum_{i \in \mathcal{N}_c} \varphi(x^i) \tilde{w}_c $$

However, it is not always possible to compute $\varphi(x^i)$. For example, the RBF kernel maps points in a space of dimension $\infty$, defined by the formula:
$$ k(x^p, x^q) =  \exp \left( - \frac{\| x^p - x^q \|^2}{2 \sigma^2} \right)$$
Fortunately, using the kernel trick, it is non necessary to calculate $\tilde{w}_c$ e $\tilde{\theta}_c$ because the decision formula is given by the following:

$$ f_{lin,min} (x) : = argmin_{c=1,...,C} \frac {|\langle\tilde{w}_c,\varphi(x)\rangle_{\mathcal{H}} + \tilde{\theta}_c |}{\|\tilde{w}_c\|_{\mathcal{H}}} \quad \quad
f_{lin,max} (x) : = argmax_{c=1,...,C} \frac {|\langle\tilde{w}_c,\varphi(x)\rangle_{\mathcal{H}} + \tilde{\theta}_c |}{\|\tilde{w}_c\|_{\mathcal{H}}}$$

where:<br><br>
$\langle\tilde{w}_c,\varphi(x)\rangle_{\mathcal{H}}= \lambda_c^T K(X_c,x)-\frac{\nu_c}{m_o}e_o^TK(X_o,x)$ <br><br>
$\tilde{\theta}_c= - \frac {1}{\|N_c\|}\sum_{i \in N_c} \left\{ K(x^i, X_c) \lambda_c - \frac{\nu_v}{m_o} K(x^i, X_o) e_o\right\}$ <br><br>
$ \|\tilde{w}_c\|^2_{\mathcal{H}} \text{ = }\langle w_c,w_c\rangle_{\mathcal{H}}\text{ = }\tilde{\lambda}_c^T K_{cc} \tilde{\lambda}_c - \frac{\nu_c}{m_o}\tilde{\lambda}^T_{c} K_{co} e_o - \frac{\nu_c}{m_o}e^T_o K_{oc} \tilde{\lambda}_{c} + \frac{\nu_c^2}{m^2_o} K_{oo} e_o  $

---

## Kernel functions

In [3]:
import numpy as np

def linear_kernel(xp, xq):
    # Linear kernel: phi(x) = x
    # Linear kernel: k(xp, xq) = xp.T xq
    return xp @ xq

def rbf_kernel(xp, xq, sigma=0.01):
    # Radial Basis Function (RBF) kernel with sigma = dev standard
    return np.exp(-np.linalg.norm(xp - xq) ** 2 / (2 * sigma ** 2))

def polynomial_kernel(xp, xq, degree=2):
    # Polynomial kernel
    return (xp @ xq + 1) ** degree

def sigmoid_kernel(xp, xq, alpha=1, constant=0):
    # Sigmoid kernel
    return np.tanh(alpha * (xp @ xq) + constant) 


---

## Gradient Descent
<span style="font-size: 14px;">

By varying the update rule for the learning rate, the SVMClass.optimize_lambda method applies the Projected Gradient Descent (PGD) in different ways, depending on two key parameters: batch_required and gamma_function. <br>
- The batch_required parameter determines how the gradient $g$ is computed, ranging from full-batch gradient descent to stochastic approaches.
The size of the mini-batch, when used, can be set explicitly as an absolute value or as a percentage of the class size. This flexibility is particularly advantageous for handling highly imbalanced datasets, where controlling the batch size proportionally helps ensure effective training.
The gradient $g$ is always projected to respect the problem constraints, ensuring feasibility at each step.
- The gamma_function parameter governs how the learning rate is updated during optimization, offering a range of strategies from constant rates to adaptive schemes like exponential or inverse decay. The specific rules for these parameters are detailed in the following tables:

parameter = batch\_required:
\begin{array}{| c | c | l |}
\hline
Value & Type & Formula \\
\hline\\
\text{none}& \text{Full GD} &g_c := \frac {\nabla \mathcal{L}}{\nabla \lambda} \\[4pt]
(-\infty..0]\\[4pt]
\hline
&& m = \text{batch\_required} \cdot m_c \\[4pt]
(0..1) & \text{Mini-batch GD} &g_c := \frac{1}{m}\sum_{x \in \mathcal{X}_c^*} \frac {\nabla \mathcal{L}_x}{\nabla \lambda} \\[4pt]
&& \mathcal{X}_c^*  \text{ randomly chosen}\quad \mathcal{X}_c^* \subseteq \mathcal{X}_c, \; |\mathcal{X}_c^*| = m \\[4pt]
\hline
&& m = 1 \\[4pt]
1 & \text{Stochastic GD} & g_c := \frac{\nabla \mathcal{L}_x}{\nabla \lambda} \text{ for } x \sim \mathcal{X}_c \\[4pt]
&& x \text{ is randomly chosen from } \mathcal{X}_c\\[4pt]
\hline
&& m = \min(\text{batch\_required, } m_c) \\[4pt]
(1..\infty) &  \text{Mini-batch GD} & g_c := \frac{1}{m}\sum_{x \in \mathcal{X}_c^*} \frac {\nabla \mathcal{L}_x}{\nabla \lambda} \\[4pt]
&& \mathcal{X}_c^*  \text{ randomly chosen}\quad \mathcal{X}_c^* \subseteq \mathcal{X}_c, \; |\mathcal{X}_c^*| = m \\[4pt]
\hline
\end{array}

parameter: gamma_func:
\begin{array}{| l | l | l | l |}
\hline
Value & Type & Learning \text{ } Rate \text{ } Update & Step\text{ } Update \\
\hline\\[4pt]
\text{gamma\_naive} & \text{Naive (Vanilla)} & \gamma := \gamma_0 & \lambda_{t+1} := \lambda_{t} - \gamma \cdot g_c \\[4pt]
\hline\\[4pt]
\text{gamma\_exponential} & \text{Exponential Decay} &  \gamma_t := \gamma_0 \cdot \exp(-\beta \cdot t) & \lambda_{t+1} := \lambda_{t} - \gamma_t \cdot g_c \\[4pt]
\hline\\[4pt]
\text{gamma\_inverse} & \text{Inverse Decay} &  \gamma_t := \frac{\gamma_0}{1 + \beta \cdot t} & \lambda_{t+1} := \lambda_{t} - \gamma_t \cdot g_c \\[4pt]
\hline\\[4pt]
\text{gamma\_newton} & \text{Newton Method} &  \gamma_t :=  H^{-1} & \lambda_{t+1} := \lambda_{t} - \gamma_t \cdot g_c \\[4pt]
\hline\\[4pt]
\text{gamma\_nag} & \text{Nesterov's Accelerated GD} & \gamma = L^{-1} &  y_{t+1} := \lambda_t - \gamma \cdot g_c \\[4pt]
&&& z_{t+1} := z_t - \frac{t+1}{2} \cdot \gamma \cdot g_c \\[4pt]
&&& \lambda_{t+1} := \frac{t+1}{t+3} y_{t+1} + \frac {2}{t+3} z_{t+1} \\[4pt]
\hline
\end{array}

In [4]:
def gamma_naive(gamma_initial, beta = None, t = None):
    return gamma_initial

def gamma_exponential(gamma_initial, beta, t):
    return gamma_initial * np.exp(-beta * t)

def gamma_inverse(gamma_initial, beta, t):
    return gamma_initial / (1 + beta * t)  # Update learning rate

'''
The gamma update for the Newton method use the SVMClass attribute self.H_inv
'''
def gamma_newton(gamma_initial, beta, t):
    return 

'''
The gamma update for the NAG method is implemented in the optimize_lambda function
'''
def gamma_nag(gamma_initial, beta, t):
    return 


---

## The SVMClass
<span style="font-size: 16px;">

This code defines the SVMClass that implements a win Parametric Margin Support Vector Machine (TPMSVM) for multi-class classification. It includes key functionalities such as computing the dual cost functions as well as verifiying the convexity of kernel matrices.<br>
The kernel matrices $k_{cc}, k_{co}, k_{oc} \text{ and } k_{oo}$ are attributes because definited by the value of c-th class and the $o$-th classes.<br><br>
The SVMClass has the following methods: <br>
- <span style="color: red;">K(X1, X2)</span>: public, returns the kernel matrix beewen set X1 and set X2
- <span style="color: red;">gradient()</span>: public, computes the full gradient for $\lambda$ or minibatch gradient (down to 1 for stochastic GD)
- <span style="color: red;">H_inv()</span>: public, return the Hessian inverse
- <span style="color: red;">J_dual()</span>: public, return the Objective function for the dual problem
- <span style="color: red;">B()</span>: public, returns the Lipschitz parameter calculated as $B = \| H\|_2$.
- <span style="color: red;">L()</span>: public, returns the smooth parameter calculated as $L = \max \text{ |eigenvalues of } H|$.
- <span style="color: red;">mu()</span>: public, returns the strong concavity parameter $\mu = \min \text{ |eigenvalues of } H|$ (only if all the eigenvalues are negative, otherwise returns 0).
- <span style="color: red;">check_convexity()</span>: public, get a msg about the convexity of $H$
- <span style="color: red;">optimize_lambda </span>:(gamma_func, max_iterations, gamma_0, beta, tolerance): public, calculate the optimum $\~{\lambda}$ for the dual problem using:
  - gamma_func for $\gamma$ update rule
  - max_iteration as max number of GD steps
  - gamma_0 as initial value of $\gamma$
  - beta as updating parameter fo $\gamma$
  - tolerance as value for breaking the GD loop
- <span style="color: red;">project(λ)</span>: public, applies the contraints at each step, thus restricting te search for the optimal solution to feasible value
- <span style="color: red;">compute_w_theta()</span>: public, compute $\~{w}$ and $\~{\theta}$ on basis of $\~{\lambda}$
- <span style="color: red;">predict(X)</span>: public, returns the predictions on set X using $\~{w}$ and $\~{\theta}$

In [5]:
class SVMClass:
    def __init__(self, class_label, X_train, Y_train, nu_c=1, alpha_c=1):
        self.class_label = class_label  # Label class
        self.X_c = X_train[Y_train == class_label]  # Subset class observations
        self.X_o = X_train[Y_train != class_label]  # Subset other observations
        self.m_c = self.X_c.shape[0]  # Sample number of the class
        self.m_o = self.X_o.shape[0]  # Sample number of the other classes
        self.Y_c = np.ones(self.X_c.shape[0])       # Labels for class observations (+1)
        self.Y_o = -1 * np.ones(self.X_o.shape[0])  # Labels for other observations (-1)
        self.n = X_train.shape[1]  # Features number
        self.Nc = 0 # number of support vectors
        # kernel type and kernel matrix
        self.kernel_type = rbf_kernel # Kernel type
        kernel_params = 0.01 #sigma for rbf kernel
        self.kernel_params = kernel_params  # Allow overriding with custom parameters
        self.Kcc = self.K(self.X_c, self.X_c)  # K(X_c, X_c)
        self.Kco = self.K(self.X_c, self.X_o)  # K(X_c, X_o)
        self.Koc = self.K(self.X_o, self.X_c)  # K(X_o, X_c)
        self.Koo = self.K(self.X_o, self.X_o)  # K(X_o, X_o)
        self.Kc = np.block([[self.Kcc, self.Kco], [self.Koc, self.Koo]])  # Kernel matrix
        self.H = - self.Kcc   
        # Initialize the variables
        self.w_c = np.zeros(self.n)  # Weight vector
        self.theta_c = 0  # Bias
        self.xi_c = np.zeros(self.m_c)      # Slack vector
        self.lambda_c = np.zeros(self.m_c)  # Lagrangian vector
        self.e_c = np.ones(self.m_c)        # e vector for class observations
        self.e_o = np.ones(self.m_o)        # e vector for other observations
        self.alpha_c = alpha_c  # Alpha parameter
        self.nu_c = nu_c     # Nu parameter
        # Optimal values
        self.optimal_lambda_c = np.zeros(self.m_c)
        self.optimal_w = np.zeros(self.n)
        self.optimal_theta = 0

    def K(self, X_1, X_2):
        # Compute the kernel matrix
        if self.kernel_type == rbf_kernel:
            return np.array([[self.kernel_type(x1, x2, 
                sigma=self.kernel_params) for x2 in X_2] for x1 in X_1])
        elif self.kernel_type == polynomial_kernel:
            return np.array([[self.kernel_type(x1, x2, 
                degree=self.kernel_params) for x2 in X_2] for x1 in X_1])
        elif self.kernel_type == sigmoid_kernel:
            return np.array([[self.kernel_type(x1, x2, 
                alpha=self.kernel_params, 
                ) for x2 in X_2] for x1 in X_1])
        else: # Linear kernel as default
            return np.array([[self.kernel_type(x1, x2) for x2 in X_2] for x1 in X_1])
    
    def phi(self, X):
        # Compute the mapping of the input data
        tmp = np.array([self.kernel_type(x, self.X_c) for x in X])
        return tmp[:, np.newaxis]
       
    def gradient(self, batch_indices=None):
        # Compute the gradient of J_dual = - Kcc lambda_c + nu_c/m_o K_oc^T e_o
        # only for the samples in batch_indices (stochastic gradient)
        if batch_indices is None: # Full gradient
            batch_indices = np.arange(self.m_c)  # Use all the indices
            normalize = False
        else:
            normalize = True
        Kcc_batch = self.Kcc[np.ix_(batch_indices, batch_indices)]
        Koc_batch = self.Koc[:,batch_indices]
        lambda_c_batch = self.lambda_c[batch_indices]
        grad_lambda = - Kcc_batch @ lambda_c_batch + (self.nu_c / self.m_o) * Koc_batch.T @ self.e_o
        if normalize:
            grad_lambda = grad_lambda /len(batch_indices)
        return grad_lambda
    
    def H_inv(self):
        try:
            # Compute the inverse of the Hessian matrix
            H_inv = np.linalg.inv(self.H)
        except np.linalg.LinAlgError:
            # Compute the pseudo-inverse of the Hessian matrix
            H_inv = np.linalg.pinv(self.H)
        return H_inv
    
    def J_dual(self):
        '''Compute the dual cost function used by CVXpy'''
        # Cost part that involves only the class observations
        dual_term1 = -0.5 * (self.lambda_c.T @ self.Kcc @ self.lambda_c)
        # Cost part that involves both class and other observations
        dual_term2 = (self.nu_c / self.m_o) * (self.e_o.T @ self.Koc @ self.lambda_c)
        # Dual cost function
        return dual_term1 + dual_term2
    
    def B(self):
        # Compute the Lipschitz constant as Frobenius norm of the Hessian matrix
        B = np.linalg.norm(self.H, 'fro')
        return B
    
    def L(self):
        # Compute the smooth constant as the maximum eigenvalue of Hessian matrix
        eigenvalues = np.linalg.eigvals(self.H)
        if np.all(eigenvalues < 0):
            # Ensure mu is positive for strong concavity
            L = max(np.abs(eigenvalues)) 
        else:
            L = 0
        return L
    
    def mu(self):
        # Compute the mu parameter as the minimum eigenvalue of Hessian matrix
        eigenvalues = np.linalg.eigvals(self.H)
        if np.all(eigenvalues < 0):
            # Ensure mu is positive for strong concavity
            mu = min(np.abs(eigenvalues)) 
        else:
            mu = 0
        return mu
    
    def project(self, lambda_new):
        # Projection of lambda_new to the feasible set
        lambda_new = np.clip(lambda_new, 0, self.alpha_c / self.m_c)  # lambda_c <= alpha_c/m_c
        total_sum = np.sum(lambda_new)
        if total_sum > self.nu_c: # e^T_c lambda_c = nu_c Constrain Normalization
            lambda_new *= self.nu_c / total_sum # Normalize lambda_c
        return lambda_new
       
    def optimize_lambda(self, batch_required=None, gamma_func=gamma_naive, max_iterations=500, 
                        gamma_0=5e-3, beta=1e-3, tolerance=1e-6, verbose=True):
        if batch_required == None or batch_required <=0:
            stochastic = False
        else: # Test if is possible to use the SGD
            if batch_required >0 and batch_required < 1:
                batch_size = math.ceil(self.m_c * batch_required)
            else:
                batch_size = np.min([self.m_c, math.ceil(batch_required)])
            full_grad_lambda = self.gradient() # Full gradient
            batch_indices = np.random.choice(self.m_c, batch_size, replace=False) 
            batch_grad_lambda = self.gradient(batch_indices)
            test = np.allclose(batch_grad_lambda, full_grad_lambda[batch_indices])
            stochastic = test
        # end if stochastic
        if verbose and stochastic:
            print(f"- Stochastic Gradient Descent")        
            print(f"- Batch size: {batch_size}")
        elif verbose:
            print(f"- Full Gradient Descent")
            print(f"- Batch size: {self.m_c}")
        # end if verbose
        if gamma_func == gamma_newton and verbose:
            print(f"- Newton method for gamma update")
            print(f"- Initial gradient norm: {np.linalg.norm(self.gradient()):.5f}")
        elif gamma_func == gamma_nag and verbose:
            print(f"- Nesterov Accelerated Gradient method")
            print(f"- Initial gradient norm: {np.linalg.norm(self.gradient()):.5f}")
        elif verbose:
            print(f"- gamma function: {gamma_func.__name__}")
            print(f"- Initial gamma: {gamma_0:.5f}")
        # end if verbose
        grad_history   = [] # History of the gradient
        lambda_history = [] # History of the lambda values
        gamma_history  = [] # History of the gamma values
        # Start the PGD algorithm
        gamma = gamma_0
        self.lambda_c = np.random.default_rng(42).random(self.m_c)
        L = self.L() # Lipschitz constant
        z = np.zeros_like(self.lambda_c) # Initialize z = 0  
        y = self.lambda_c.copy() # Initialize y = lambda_c
        for t in range(0, max_iterations + 1):
            if stochastic: # Use the stochastic gradient
                batch_indices = np.random.choice(self.m_c, batch_size, replace=False) 
                grad_lambda = self.gradient(batch_indices)
                grad_lambda = np.reshape(grad_lambda, self.lambda_c[batch_indices].shape)
                if gamma_func == gamma_newton: # Update the lambda values using the Newton method
                    H_inv_batch = self.H_inv()[np.ix_(batch_indices, batch_indices)]
                    update = (H_inv_batch @ grad_lambda).reshape(self.lambda_c[batch_indices].shape)
                    self.lambda_c[batch_indices] = self.lambda_c[batch_indices] - update
                elif  gamma_func == gamma_nag: # Update the lambda using the NAG
                    y[batch_indices] = self.lambda_c[batch_indices] - 1/L * grad_lambda
                    z[batch_indices] = z[batch_indices] - (t + 1) / (2 /L) * grad_lambda
                    self.lambda_c[batch_indices] = (t + 1) / (t + 3) * y[batch_indices] + 2 / (t + 3) * z[batch_indices]
                else: # Update the lambda values using scalar eta
                    gamma = gamma_func(gamma_0, beta, t)
                    self.lambda_c[batch_indices] = self.lambda_c[batch_indices] - gamma *grad_lambda               
            else: # Use the full gradient
                grad_lambda = self.gradient()
                if gamma_func == gamma_newton: # Update the lambda values using the Newton method
                    self.lambda_c = self.lambda_c - self.H_inv() @ grad_lambda
                elif  gamma_func == gamma_nag: # Update the lambda using the NAG
                    y = self.lambda_c - 1/L * grad_lambda
                    z = z - (t + 1) / (2 /L) * grad_lambda
                    self.lambda_c = (t + 1) / (t + 3) * y + 2 / (t + 3) * z
                else: # Update the lambda values using scalar eta
                    gamma = gamma_func(gamma_0, beta, t)
                    self.lambda_c = self.lambda_c - gamma * grad_lambda 
            # end if stochastic
            self.lambda_c = self.project(self.lambda_c) # Project lambda_new and save it
            grad_history.append(np.linalg.norm(grad_lambda))
            lambda_history.append(np.linalg.norm(self.lambda_c))
            gamma_history.append(gamma)
            if np.linalg.norm(grad_lambda) < tolerance: break
        # end for       
        if verbose:
            print(f"- Optimization completed in {t} iterations")
            print(f"- Final gradient norm: {np.linalg.norm(grad_lambda):.5f}")
        return(grad_history, lambda_history, gamma_history)
    
    def compute_w_theta(self, verbose=True):
        # Support vector individuation
        nonpositive_indices = np.where(self.lambda_c <= 0)[0]
        over_indices = np.where(self.lambda_c >= self.alpha_c / self.m_c)[0]
        sv_indices = np.setdiff1d(np.arange(len(self.lambda_c)), np.union1d(nonpositive_indices, over_indices))
        self.Nc = len(sv_indices) # Number of support vectors
        if self.Nc == 0 and verbose:
            print("- No support vectors found")
            return np.zeros(self.X_c.shape[0])  # Return neutral prediction
        elif self.Nc == 0:
            return np.zeros(self.X_c.shape[0]) # Return neutral prediction
        elif verbose:
            print(f"- Support vectors found: {len(sv_indices)}")
        # Set lambda_c = 0 for non-active Lagrange multipliers (non-support vectors)    
        self.optimal_lambda_c[nonpositive_indices] = 0  # set to 0 for nonpositive_indices
        self.optimal_lambda_c[over_indices] = 0         # set to 0 for over_indices
        if self.kernel_type == linear_kernel:
            # Compute the optimal w and theta only for the linear kernel
            w = self.X_c.T @self.optimal_lambda_c - (self.nu_c / self.m_o) * self.X_o.T @ self.e_o 
            self.optimal_w = np.array(w).astype(np.float64) 
            self.optimal_theta = - 1/self.Nc * np.sum(self.X_c @ self.optimal_w)        
        return

    def check_convexity(self):
        # Primal problem = minimum => H convex
        # Dual problem = maximum => H concave
        # Eigenvalues of H must be <= 0 for convexity
        # H concave => Kcc convex
        eigenvalues = np.linalg.eigvals(self.H)
        positive_count = np.sum(eigenvalues > 0)
        negative_count = np.sum(eigenvalues < 0)
        zero_count = np.sum(eigenvalues == 0)
        print(f"- Number of Eigenvalues for H: Positive = {positive_count}, Negative = {negative_count}, Null = {zero_count}")
        if negative_count == len(eigenvalues):  # All eigenvalues are negative
            print("- Hessian matrix is Negative Definite: objective is Strictly Concave")
        elif positive_count==0:  # No positive eigenvalues
            print("- Hessian matrix is Negative Semi-Definite: objective is Concave")
        elif positive_count == len(eigenvalues):  # All eigenvalues are positive
            print("- Hessian matrix is Positive Definite: objective is Strictly Convex")
        elif negative_count==0:  # No negative eigenvalues
            print("- Hessian matrix is Positive Semi-Definite: objective is Convex")
        else:
            print("- Hessian matrix is Indefinite: Neither convex nor concave")
        #end if
        return np.all(eigenvalues <= 0), self.H
    
    def predict(self, X):
        Kcx = self.K(self.X_c, X)  # Kernel beetwen X_c e X (K(X_c, X))
        Kox = self.K(self.X_o, X)  # Kernel beetwen X_o e X (K(X_o, X))
        Kxc = self.K(X, self.X_c)  # Kernel beetwen X e X_c (K(X, X_c))
        Kxo = self.K(X, self.X_o)  # Kernel beetwen X e X_o (K(X, X_o))
        if self.Nc == 0:
            print(f"- Warning: Nc is zero for class {self.class_label}. Returning neutral prediction.")
            return np.zeros(X.shape[0])  # Return neutral prediction
        w_per_x = (self.optimal_lambda_c.T @ Kcx) - (self.nu_c / self.m_o) * (self.e_o.T @ Kox)
        theta_1 = - (1/self.Nc) * Kxc @ self.optimal_lambda_c 
        theta_2 = - (self.nu_c / self.m_o) * Kxo@ self.e_o
        theta = theta_1 + theta_2
        w2_1 = (self.optimal_lambda_c.T @ self.Kcc @ self.optimal_lambda_c)
        w2_2 = - (self.nu_c / self.m_o) * (self.optimal_lambda_c.T @ self.Kco @ self.e_o)
        w2_3 = - (self.nu_c / self.m_o) * (self.e_o.T @ self.Koc @ self.optimal_lambda_c)
        w2_4 = (self.nu_c ** 2 / self.m_o ** 2) * (self.e_o.T @ self.Koo @ self.e_o)
        norm_w = np.sqrt(w2_1+w2_2+w2_3+w2_4)
        return (w_per_x + theta) / norm_w


---

## Other functions
<span style="font-size: 14px;">

The SVMClass contains all the code usefull for defined the optimal hyperplane thats separe one class from the other, but to complete the model it's necessary to provide some other usefull function:
- <span style="color: red;">decision_function</span>: makes predictions based on the trained SVM models, enhancing modularity for multi-class classification. This structure makes it flexible for extensions and adapts well to different optimization and classification tasks.<br>
- <span style="color: red;">validate_model</span>: validates the multiclass model returning the accuracy on the validation data set. The optimize_hyper use this function on subfolder of training data set. The final validation is performed on validation data set <br>
- <span style="color: red;">optimize_hyper</span>: that using the cross-validation technique (5 fold) on training data set computes the best value for the hyperparameters $\alpha_c$ and $nu_c$ for each class.<br>
- <span style="color: red;">my_plot</span>: avoids code duplication for the plots

In [6]:
def decision_function(X, models):
    # Compute the decision function for a given input X
    all_predictions = []
    for model in models:
        prediction = model.predict(X)
        if np.all(prediction == 0):
            prediction = np.full(X.shape[0], model.class_label) # the first class is default
        all_predictions.append(prediction)
    all_predictions = np.array(all_predictions)
    predictions_min = np.argmin(all_predictions, axis=0) # Find the class with the minimum value 
    predictions_max = np.argmax(all_predictions, axis=0) # Find the class with the maximum value
    return np.array(predictions_min), np.array(predictions_max) # Return the predictions chosen by the min and max values

from sklearn.model_selection import KFold
from sklearn.metrics import classification_report
def validate_model(classes, class_mapping, models, X_val, Y_val,  print_report=True):
    # Validate the model on the validation set
    predictions_min, predictions_max = decision_function(X_val, models)
    predictions = predictions_max
    if all(isinstance(v, int) for v in class_mapping.values()):
        # Class labels are integers
        target_names = [f"Class {label}" for label in classes]
    else:
        # Class labels are strings
        inverse_class_mapping = {v: k for k, v in class_mapping.items()}
        target_names = [inverse_class_mapping[label] for label in classes]
    # Compute the classification report
    report = classification_report(
        Y_val,
        predictions,
        labels=classes,
        target_names=target_names,
        zero_division=0,
        output_dict=True
    )
    accuracy = report.get('accuracy', None)
    if accuracy is None:
        accuracy = 0  # Set accuracy to 0 if not found
    if print_report:
        print(f"Validation accuracy: {accuracy:.3f} with alpha: {models[0].alpha_c:.3f}, nu: {models[0].nu_c:.3f} and kernel param: {models[0].kernel_params:.3f}\n")
        print(pd.DataFrame(report).transpose())
    return accuracy

def my_plot(nRows, nCols, plotNum, x, y, title, xLabel, yLabel, color, figure=None):
    if figure is None:
        figure = plt.figure(figsize=(12, 6))
    y_plot = y.copy()
    if len(y) > 1 and y[0] > 10 * y[1]:
        y_plot[0] = None
    plt.subplot(nRows, nCols, plotNum)
    plt.plot(y_plot, label=yLabel, color=color)
    plt.title(title)
    plt.xlabel(xLabel)
    plt.ylabel(yLabel)
    plt.legend()
    plt.grid()
    return figure
    
def machineEpsilon():
    # Compute the maximum possible accuracy for this machine
    eps = 1.0
    while eps + 1.0 != 1.0:
        eps /= 2.0
    return eps



---

## The TPMSVM function

<span style="font-size: 14px;">

Using the previous code:
- Create an instance of SVMClass for each class in the dataset.
- For each class
  - optimize the Lagrange parameter $\lambda_c$
  - calculate the best $w_c$ and $\theta_c$
  - make the best predictions using the decision_function
  - return the accuracy on the validation data set


In [7]:
def TPMSVM(classes, # List of classes
           class_mapping, # Mapping of class labels
           X_train, Y_train, # Training set
           X_val, Y_val, # Validation set
           best_alpha, best_nu,  # Best hyperparameters
           kernel_type = rbf_kernel, # Kernel type
           kernel_params = 0.01, # Kernel parameters
           batch_required = None, # Full or mini-batch gradient down to 1
           gamma_func = gamma_naive, # Learning rate function
           tolerance = 100 * machineEpsilon(), #
           T = 500, # Number of iterations
           verbose=False):# Verbose mode
    plt.close('all') # Close all previous plots to save memory
    models = {}
    for class_label in classes:
        myClass = SVMClass(class_label, X_train, Y_train)
        myClass.nu_c = best_nu
        myClass.alpha_c = best_alpha
        myClass.kernel_type = kernel_type  # Assign kernel type
        myClass.kernel_params = kernel_params  # Assign kernel parameters
        print(f"\nClass: {myClass.class_label}")
        #  Check if the kernel is as expected and optimizating setup
        convexity_results, H_matrix = myClass.check_convexity()
        if  not convexity_results or (batch_required != None and myClass.mu()==0): 
            # non convex function or SGD/miniBatch with non strong convex function
            # then use Lipschitz function formula
            tolerance = 1e-3
            R = myClass.alpha_c / myClass.m_c
            B = myClass.B() 
            T = R**2 * B**2 / tolerance**2
            T = np.ceil(T).astype(int)
            errore_max = R *B / np.sqrt(T)
            gamma_0 = R / (B * np.sqrt(T))
        else:
            beta=1e-3 #default value
            R = myClass.alpha_c / myClass.m_c
            gamma_0 = 1/ myClass.L()  # Initial learning rate
            log = np.log((R**2 * myClass.L()) / (2 * tolerance)) # Logarithm
            if batch_required != None:
                tolerance = 1e-3
                T = 2 * myClass.B()**2 / (myClass.mu() * tolerance);   
                T = np.ceil(T).astype(int)
                errore_max = 2* myClass.B()**2 /(myClass.mu() * (T+1))
                gamma_func = gamma_inverse
                gamma_0 = 2/(myClass.mu())
                beta = 1 
            elif gamma_func == gamma_nag: # Use the NAG formula
                T = R * np.sqrt(2 * myClass.L() / tolerance)
                T = np.ceil(T).astype(int)
                errore_max = 2 * R**2 * myClass.L() / T**2
            elif myClass.mu() != 0 and log > 0: # use the strongly convex function formula
                T = myClass.L()/myClass.mu() * log  # Number of iterations
                T = np.ceil(T).astype(int)
                errore_max = (R**2 * myClass.L() / 2) *(1- myClass.mu() / myClass.L())**T
            else: # use the smooth function formula
                T = R^2 * myClass.L() / (2 * tolerance)
                errore_max = (R**2 * myClass.L()) / (2 *T )
        # Run the optimization and store the model
        grad_history, lambda_history, gamma_history = myClass.optimize_lambda(batch_required,
                gamma_func, T, gamma_0, beta, tolerance, verbose)
        myClass.compute_w_theta(verbose) # Compute w, theta for the current class
        models[class_label] = myClass # Store the model
        # Plot if verbose
        if verbose:
            if myClass.kernel_type == linear_kernel:
                print(f"- w: {np.array2string(myClass.optimal_w, precision=5)}")
                print(f"- theta: {myClass.optimal_theta:.5f}")
            print(f"- R^2: {R**2}")
            print(f"- B  parameter: {myClass.B()}")
            print(f"- L  parameter: {myClass.L()}")
            print(f"- mu parameter: {myClass.mu()}")
            print(f"- tolerance: {tolerance:.3e}")
            print(f"- T (iterations): {T}")
            print(f"- max error: {errore_max}")
            figure = plt.figure(figsize=(12, 6))
            figure = my_plot(3, 1, 1, range(len(grad_history)), grad_history, 
                    f' {gamma_func.__name__} GD: Class {myClass.class_label} Gradient over Iterations', 'Iteration', 'Gradient', 'red', figure) 
            figure = my_plot(3, 1, 2, range(len(lambda_history)), lambda_history, 
                    f' {gamma_func.__name__} GD: Class {myClass.class_label} Lambda over Iterations', 'Iteration', 'Lambda', 'black', figure) 
            figure = my_plot(3, 1, 3, range(len(gamma_history)), gamma_history, 
                    f' {gamma_func.__name__} GD: Class {myClass.class_label} Learning Rate over Iterations', 'Iteration', 'Learning rate', 'blue', figure)
            plt.tight_layout()
            plt.show()
        # end if verbose
    # end for
    print(f"\n --- Validation Results ---\n")
    accuracy = validate_model(classes, class_mapping, list(models.values()), X_val, Y_val, verbose)
    return accuracy, models
    


---

## Hyper parameters $\alpha$ and $\mu$ setting

<span style="font-size: 14px;">

Using the 5-fold cross-validation tecnique, a grid of values for the hyper parameters $\alpha$ and $\mu$ is tested. For semplicity, all the class share the same paramenters $\alpha_c = \alpha$ and $\mu_c = \mu$. <br>
The choice is made based on the accuracy achieved with the tested values.

In [8]:
import time
import numpy as np
from skopt import gp_minimize
from skopt.space import Real, Integer
from skopt.utils import use_named_args
from sklearn.model_selection import StratifiedKFold

def evaluate_combination(classes, class_mapping,
                         X_train, Y_train,
                         alpha, nu, 
                         kernel_type, kernel_params,
                         batch_required, gamma_func, tolerance, T, 
                         verbose=False):
    # Evaluate a single combination of hyperparameters using cross-validation
    start_time = time.time()
    split_accuracies = []
    # Perform K-Fold cross-validation
    kf = StratifiedKFold(n_splits=4, shuffle=True)
    for train_index, val_index in kf.split(X_train, Y_train):
        X_train_split, X_val_split = X_train[train_index], X_train[val_index]
        Y_train_split, Y_val_split = Y_train[train_index], Y_train[val_index]
        try: # Train the model and evaluate accuracy for the split
            split_accuracy, models = TPMSVM(classes, class_mapping,
                                            X_train_split, Y_train_split, 
                                            X_val_split, Y_val_split,
                                            alpha, nu,
                                            kernel_type, kernel_params,
                                            batch_required, gamma_func, tolerance, T, 
                                            verbose=verbose)
            if split_accuracy is None: # kernel is not as expected
                return -1
            split_accuracies.append(split_accuracy)
        except ZeroDivisionError:
            print(f"Warning: ZeroDivisionError for nu_c: {nu:.3f}, alpha_c: {alpha:.3f}, kernel_params: {kernel_params:.3f}. Skipping this combination.")
            return -1
    # Calculate mean accuracy and log the results if there are valid accuracies
    if split_accuracies:
        accuracy = np.mean(split_accuracies)
        elapsed_time = time.time() - start_time
        print(f"Elapsed time: {elapsed_time:.3f} sec - alpha_c: {alpha:.3f}, nu_c: {nu:.3f}, mean accuracy: {accuracy:.3f}")
        return accuracy
    else:
        print(f"No valid accuracy for alpha_c: {alpha:.3f}, nu_c: {nu:.3f}, kernel_params: {kernel_params:.3f}")
        return -1

def optimize_hyper(classes, class_mapping, 
                X_train, Y_train, 
                default_alpha, default_nu, 
                kernel_type, default_sigma,
                batch_required, gamma_func, tolerance, T,
                file_name, 
                method='none', 
                verbose=False ):
    if method == 'none':
        return default_alpha, default_nu, default_sigma
    # Hyperparameter grid boundaries and steps
    ALPHA_MIN, ALPHA_MAX, ALPHA_STEP = -7.0, 9.0, 2.0
    K_MIN, K_MAX, K_STEP = 0.1, 0.9, 0.4
    J_MIN, J_MAX, J_STEP = -7.0, 4.0, 1.0
    results_df = pd.DataFrame(columns=['alpha', 'nu', 'sigma', 'accuracy'])
    alpha_values = np.arange(ALPHA_MIN, ALPHA_MAX + ALPHA_STEP, ALPHA_STEP)
    k_values = np.arange(K_MIN, K_MAX + K_STEP, K_STEP)
    j_values = np.arange(J_MIN, J_MAX + J_STEP, J_STEP)
    if method == 'grid':  # Grid Search
        for j in j_values:
            sigma = 2.0 ** j
            for alpha in alpha_values:
                for k in k_values:
                    nu = k * alpha  # Calculate nu based on alpha and k
                    kernel_params = sigma # Kernel parameters
                    accuracy = evaluate_combination(classes, class_mapping,
                                                    X_train, Y_train,
                                                    alpha, nu,
                                                    kernel_type, kernel_params,
                                                    batch_required, gamma_func, tolerance, T, 
                                                    verbose=verbose)
                    if accuracy != -1: # Log the results if valid
                        results_df.loc[len(results_df)] = {'alpha': alpha, 'nu': nu, 'sigma': sigma, 'accuracy': accuracy}
    elif method == 'bayesian':  # Bayesian Optimization
        space = [
            Real(ALPHA_MIN, ALPHA_MAX, name="alpha"),
            Real(K_MIN, K_MAX, name="k"),
            Integer(J_MIN, J_MAX, name="j")
        ]
        @use_named_args(space) # Use the named arguments
        def objective(alpha, k, j):
            nu = k * alpha
            sigma = 2.0 ** j
            accuracy = evaluate_combination(classes, class_mapping,
                                            X_train, Y_train,
                                            alpha, nu,
                                            kernel_type, sigma,
                                            batch_required, gamma_func, tolerance, T, 
                                            verbose=verbose)
            if accuracy == -1:
                return 1e6 # Return a large value for invalid accuracy
            else:
                results_df.loc[len(results_df)] = {'alpha': alpha, 'nu': nu, 'sigma': sigma, 'accuracy': accuracy}
                return -accuracy  # Return the negative accuracy for minimization
        gp_minimize(objective, dimensions=space, n_calls=100, random_state=42)
    #end if
    results_df.to_csv(f"{file_name}.csv", index=False)
    max_row = results_df.loc[results_df['accuracy'].idxmax()]
    best_nu    = max_row['nu']
    best_alpha = max_row['alpha']
    best_sigma = max_row['sigma']
    best_accuracy = max_row['accuracy']
    print(f"Best parameters: alpha={best_alpha:.5f}, nu={best_nu:.5f}, sigma={best_sigma:.5f} ,accuracy={best_accuracy:.5f}")
    return best_nu, best_alpha, best_sigma



---

## Hyperparameters Sensitivity Plot

In [9]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from matplotlib import cm
from scipy.interpolate import griddata

def plot_hp_data(file_name="accuracy_results", title= None, best_sigma=None):
    # load the hyperparameter data
    hp_data_df = pd.read_csv(file_name, sep=',')
    hp_data_df.columns = ['alpha', 'nu', 'sigma','accuracy']
    # Clean the data and sort it
    hp_data_df['alpha'] = hp_data_df['alpha'].astype(float)
    hp_data_df['nu'] = hp_data_df['nu'].astype(float)
    hp_data_df['sigma'] = hp_data_df['sigma'].astype(float)
    hp_data_df['accuracy'] = hp_data_df['accuracy'].astype(float)
    if best_sigma is not None:
        hp_data_df = hp_data_df.loc[hp_data_df['sigma'].round(3)  == best_sigma]
    hp_data_df = hp_data_df.sort_values(by=['nu', 'alpha']).reset_index(drop=True)
    # Names of the columns
    nu = hp_data_df['nu'].values
    alpha = hp_data_df['alpha'].values
    sigma = hp_data_df['sigma'].values
    accuracy = hp_data_df['accuracy'].values
    # Create a grid for the interpolation
    nu_lin = np.linspace(np.min(nu), np.max(nu), 500)
    alpha_lin = np.linspace(np.min(alpha), np.max(alpha), 500)
    nu_grid, alpha_grid = np.meshgrid(nu_lin, alpha_lin)
    # Interpolate the data with a linear method
    accuracy_grid = griddata((nu, alpha), accuracy, (nu_grid, alpha_grid), method='linear')
    # Find the maximum point
    max_point_idx = np.argmax(accuracy)
    nu_max_real, alpha_max_real, sigma_max_real, accuracy_max_real = nu[max_point_idx], alpha[max_point_idx], sigma[max_point_idx], accuracy[max_point_idx]
    # Plot
    fig = plt.figure(figsize=(10, 6))
    ax = fig.add_subplot(111, projection='3d')
    # Add the surface plot
    surface = ax.plot_surface(nu_grid, alpha_grid, accuracy_grid, cmap=cm.viridis, 
                              edgecolor='none', alpha=0.7)
    # Add the scatter plot
    ax.scatter(nu, alpha, accuracy, color='black', s=3, zorder=5)
    ax.scatter(nu_max_real, alpha_max_real, accuracy_max_real, color='red', s=50, 
               label=f'Max Accuracy: {accuracy_max_real:.4f} (alpha: {alpha_max_real:.3f}, nu: {nu_max_real:.3f}, sigma: {sigma_max_real:.3f})', zorder=10)
    # Get the limits of the axes
    ax.set_xlim(np.min(nu), np.max(nu))
    ax.set_ylim(np.min(alpha), np.max(alpha))
    ax.set_zlim(np.min(accuracy), np.max(accuracy))
    # Add the lines for the maximum point
    ax.plot([nu_max_real, nu_max_real], [alpha_max_real, alpha_max_real], 
            [np.min(accuracy), accuracy_max_real], color='red', linestyle='-', linewidth=1, zorder=20)
    ax.plot([nu_max_real, nu_max_real], [np.min(alpha), alpha_max_real], 
            [np.min(accuracy), np.min(accuracy)], color='red', linestyle='-', linewidth=1, zorder=20)
    ax.plot([nu_max_real, np.max(nu)], [alpha_max_real, alpha_max_real], 
            [np.min(accuracy), np.min(accuracy)], color='red', linestyle='-', linewidth=1, zorder=20)
    # Add legend and labels
    fig.colorbar(surface, shrink=0.5, aspect=5)
    ax.legend()
    ax.set_xlabel('nu')
    ax.set_ylabel('alpha')
    ax.set_zlabel('accuracy')
    if title is None:
        ax.set_title('Hyperparameter Sentivity Plot')
    else:
        ax.set_title('Hyperparameter Sentivity Plot' + title)
    plt.show()
    # Create the pivot table
    accuracy_pivot = hp_data_df.pivot_table(index='accuracy', aggfunc='size').reset_index(name='count')
    accuracy_pivot = accuracy_pivot.sort_values(by='accuracy', ascending=False).reset_index(drop=True)
    total_count = accuracy_pivot['count'].sum()
    total_row = pd.DataFrame([['Total', total_count]], columns=['accuracy', 'count'])
    accuracy_pivot = pd.concat([accuracy_pivot, total_row], ignore_index=True)
    print("Pivot table of accuracy values and their counts:")
    print(accuracy_pivot)
    return



---

## Using CVXpy
<span style="font-size: 14px;">

The following code implements a multi-class classification system based on TPMSVM, using **cvxpy** to solve the dual optimization problem for each class, 
- The  <span style="color: red;">create_hyperplane</span> function defines the SVM dual problem, optimizing the Lagrange multipliers $\lambda_c$ under specific constraints, including normalization and value limits. The **cvxpy** library is employed to maximize the dual objective function while ensuring that the constraints are satisfied. 
- The <span style="color: red;">create_model</span> function builds a PTMSVM model which consists of a list of optimized hyperplanes using the best hyperparameters. While the primary model is already implemented, **cvxpy** is used here to solve the optimization problem as a benchmark for comparing the results of my custom optimization code. 
- The <span style="color: red;">validate_model</span> function, called by the create_model, evaluates the model by selecting the best class based on the decision function and calculates the accuracy of the trained models on the validation data.
</span>

In [10]:
import cvxpy as cp
import numpy as np
import warnings
import copy

def create_hyperplane(myClass):
    try:
        myClass.lambda_c = cp.Variable(myClass.m_c, nonneg=True)  # Dual variable >= 0
        myClass.xi_c = cp.Parameter(myClass.m_c, nonneg=True)  # Slack variable >= 0
        objective = myClass.J_dual()  # Dual cost function
        constraints = [
            cp.sum(myClass.lambda_c) == myClass.nu_c,  # e^T_c lambda_c = nu_c Constrain Normalization
            myClass.lambda_c <= myClass.alpha_c / myClass.m_c  # lambda_c <= alpha_c/m_c Constrain Limitation
        ]
        problem = cp.Problem(cp.Maximize(objective), constraints)  # Maximize J^*(lambda_c) with constraints
        with warnings.catch_warnings():
            warnings.simplefilter("ignore")
            problem.solve()
        if problem.status != cp.OPTIMAL:  # Check if the solution is not optimal
            print(f"Problem status: {problem.status} for class {myClass.class_label}. Continuing with next class.")
            return problem.status, None, None  # Continue without optimal results
        myClass.lambda_c = np.array(myClass.lambda_c.value)  # Convert to a NumPy array 
        myClass.compute_w_theta(True) # Compute w, theta for the current class
        return problem.status, myClass.optimal_w, myClass.optimal_theta
    except cp.error.DCPError as e:
        print(f"An error occurred: Problem is not convex! {str(e)}")
        return "Not Convex", None, None  # Return a status instead of exiting
    except cp.SolverError as e:
        print(f"An error occurred: The solver could not find a solution. {str(e)}")
        return "Solver Error", None, None  # Return a status instead of exiting
    except Exception as e:
        print(f"An unexpected error occurred: {str(e)}")
        return "Unexpected Error", None, None  # Return a status instead of exiting

def create_model(classes, class_mapping, X_val, Y_val, models):
    # Create a copy of the models and set the best hyperparameters
    print(f"\n --- Optimization Results with CVXpy ---")
    models_copy = {class_label: copy.deepcopy(model) for class_label, model in models.items()}
    for class_label, model in models_copy.items():
        print(f"\nClass: {class_label}")
        model.lambda_c = np.random.default_rng(42).random(model.m_c)  # Reinitalize lambda_c
        esito, model.optimal_w, model.optimal_theta = create_hyperplane(model)
        print(f"- Problem status: {esito}")
    print(f"\n --- Validation Results with CVXpy ---\n")
    # Validate the model on the validation set
    validate_model(classes, class_mapping, list(models_copy.values()), X_val, Y_val)
    return


---

## Using sklearn.SVM

<span style="font-size: 14px;">

This code demonstrates a basic implementation of a One-vs-Rest (OvR) classification strategy using scikit-learn's OneVsRestClassifier with a Support Vector Classifier (SVC) and a RBF kernel. The dataset is first split into training and validation sets, after which the classifier is trained on the training set and tested on the validation set. The final accuracy and a detailed classification report are then printed to assess the model's performance.

It's important to note that, in this setup, we have no control over the inner workings of the model itself beyond choosing the linear SVC as the base classifier. Therefore, this implementation can be used as a black-box benchmark to compare against the TPMSVM developed.
</span>

In [11]:
from sklearn.multiclass import OneVsRestClassifier
from sklearn.svm import SVC 
from sklearn.metrics import classification_report, accuracy_score
def use_sklearnSVM(X_train, Y_train, X_val, Y_val):
  ovr_classifier = OneVsRestClassifier(SVC(kernel='rbf'))
  ovr_classifier.fit(X_train, Y_train) # Train the classifier
  predictions = ovr_classifier.predict(X_val) # Make predictions on the validation set
  accuracy_SVM = accuracy_score(Y_val, predictions) # Compute the final accuracy
  print(f"\n --- Validation Results with OVRC by sklearn --- \n")
  print(f"Validation Accuracy: {accuracy_SVM:.5f}\n")
  report = classification_report(Y_val, predictions) # Print the classification report
  print(report)
  return


---

## Using sklearn.CART

<span style="font-size: 14px;">

This code demonstrates a basic implementation of a CART-based classifier using scikit-learn's DecisionTreeClassifier with the default Gini impurity criterion. The dataset is first split into training and validation sets, after which the decision tree is trained on the training set and tested on the validation set. The final accuracy and a detailed classification report are printed to evaluate the model's performance.

In this implementation, the maximum depth of the tree is fixed at 10 to prevent overfitting and ensure better generalization. This depth limit controls the complexity of the tree and helps to avoid overfitting to noise in the dataset.

Unlike the OvR SVM implementation, the decision tree model here is fully interpretable, allowing us to examine the tree structure and understand the decision-making process. However, it does not leverage the concept of hyperplanes or kernel transformations as in TPMSVM. This makes CART a strong baseline for comparison, particularly in scenarios where model interpretability is prioritized over non-linear separability.
</span>

In [12]:
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import accuracy_score, classification_report

def use_sklearnCART(X_train, Y_train, X_val, Y_val):
    # Setting th4e model CART
    cart_model = DecisionTreeClassifier(
        criterion='gini',  # 'entropy' is the other option
        max_depth=10,    # Max tree depth
        random_state=42
    )

    cart_model.fit(X_train, Y_train)
    predictions = cart_model.predict(X_val)
    accuracy_cart = accuracy_score(Y_val, predictions)
    print(f"\n --- Validation Results with CART by sklearn --- \n")
    print(f"Validation Accuracy: {accuracy_cart:.5f}\n")
    print(classification_report(Y_val, predictions))
    n_leaves = cart_model.tree_.n_leaves
    n_nodes = cart_model.tree_.node_count - n_leaves
    print(f"Number of partition nodes: {n_nodes}")
    print(f"Number of leaves nodes: {n_leaves}")

    return cart_model


---
<span style="font-size: 12px;">

- Accuracy: The proportion of correct predictions relative to the total predictions.<br>
- Precision: The proportion of true positives relative to the total predicted positives. It indicates how reliable the positive predictions are.<br>
- Recall: The proportion of true positives relative to the total actual positives. It indicates how well the model finds the positives.<br>
- F1 Score: The harmonic mean of precision and recall. It is useful when there is an imbalance between classes.<br>